In [1]:
# -*- coding: utf-8 -*-
import os
import re
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import xgboost as xgb
import jieba

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download("punkt_tab")

/home/b10173209/venv/lib/python3.8/site-packages/xgboost/core.py:265: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc 2.28+) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
[nltk_data] Downloading package punkt to /home/b10173209/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/b10173209/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/b10173209/nltk_data...
[nltk_data]   Package punkt_tab is alrea

True

In [2]:
# 定義主目錄
main_folder = '/home/b10173209/test.file/SS/class10(KFold)'

# 分類的清單
categories = ["anger","fear","joy","love","sadness","surprise"]

# 自定義 Tokenizer
def custom_tokenizer(text):
    tokens = jieba.lcut(text)
    return tokens

# 定義文本預處理函數
def preprocess_text(text):
    processed_text = text.lower()  # 將文本轉為小寫
    processed_text = re.sub(r'[^\w\s]', '', processed_text)  # 去除標點符號
    return processed_text


In [6]:

# 定義參數網格
param_grid = {'learning_rate': [0.1],
              'max_depth': [9],
              'n_estimators': [300]}

# 初始化 category_results 列表
all_validation_results = []


In [7]:
for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
        for n_estimators in param_grid['n_estimators']:
            print(f'''
##################################################################################
##     Start Running learning_rate : {learning_rate}、max_depth : {max_depth}、n_estimators : {n_estimators}
################################################################################## '''
            )

            # 遍歷每個類別
            for category in categories:
                category_folder = os.path.join(main_folder, category)

                # 遍歷每個子資料夾
                for i in range(5):
                    subfolder = f'{category}_{i}'
                    subfolder_path = os.path.join(category_folder, subfolder)

                    # Debugging: Print folder path
                    print(f'資料夾路徑: {subfolder_path}')

                    # 檢查資料夾是否存在
                    if not os.path.exists(subfolder_path):
                        print(f'資料夾不存在: {subfolder_path}')
                        continue

                    # 讀取訓練集
                    train_file = f'train_{category}_{i}.csv'
                    train_df = pd.read_csv(os.path.join(subfolder_path, train_file))

                    # 讀取測試集
                    test_file = f'test_{category}_{i}.csv'
                    test_df = pd.read_csv(os.path.join(subfolder_path, test_file))

                    # 預處理文本數據
                    train_df['text'] = train_df['text'].apply(preprocess_text)
                    test_df['text'] = test_df['text'].apply(preprocess_text)

                    # 訓練集的標籤轉換為數字
                    train_labels = train_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()
                    test_labels = test_df[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                    # 使用 StratifiedKFold 進行分層交叉驗證
                    num_folds = 5
                    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

                    # 遍歷每一折
                    for fold, (train_index, val_index) in enumerate(skf.split(train_df, train_labels)):
                        print(f"折數: {fold + 1}/{num_folds}")

                        # 分割訓練和驗證集
                        train_data, val_data = train_df.iloc[train_index], train_df.iloc[val_index]

                        # 提取標籤
                        train_labels_fold = train_data[category].tolist()
                        
                        # 計算正類別與負類別的比例
                        positive_class_count = sum(train_labels_fold)  # 正類別數量 (標籤為1)
                        negative_class_count = len(train_labels_fold) - positive_class_count  # 負類別數量 (標籤為0)
                        
                        # 避免除以零的情況
                        if positive_class_count == 0:
                            scale_pos_weight = 0  # 無正類別時，設定權重為0
                        else:
                            scale_pos_weight = negative_class_count / positive_class_count  # 類別權重

                        # 訓練集的標籤轉換為數字
                        train_labels_fold = train_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                        # 驗證集的標籤轉換為數字
                        val_labels_fold = val_data[category].apply(lambda x: 1 if x == 1 else 0).tolist()

                        # 建立 TfidfVectorizer 將文本數據轉換為 TF-IDF 特徵
                        vectorizer = TfidfVectorizer(tokenizer=custom_tokenizer, max_features=None)
                        X_train_fold = vectorizer.fit_transform(train_data['text'])
                        X_val_fold = vectorizer.transform(val_data['text'])
                        X_test = vectorizer.transform(test_df['text'])

                        # 初始化 XGBoost 分類器
                        xgb_model = xgb.XGBClassifier(learning_rate=learning_rate, max_depth=max_depth, n_estimators=n_estimators, scale_pos_weight=scale_pos_weight, random_state=42)

                        # 訓練 XGBoost 模型
                        xgb_model.fit(X_train_fold, train_labels_fold)

                        # 在驗證集上進行預測
                        predictions_val = xgb_model.predict(X_val_fold)

                        # 在測試集上進行預測
                        predictions_test = xgb_model.predict(X_test)

                        # 將原始答案、預測結果、id 和捐款留言存入 DataFrame
                        validation_results_df = pd.DataFrame({
                            'Folder': [f'{subfolder}_{fold + 1}'] * len(val_data),
                            'Fold': [fold + 1] * len(val_data),
                            'id': val_data['id'].tolist(),
                            'Labels': val_labels_fold,
                            'Predictions': predictions_val,
                            '捐款留言': val_data['text'].tolist()
                        })

                        # 將每個折數的結果存入列表
                        all_validation_results.append(validation_results_df)

# 合併所有折數的結果
all_validation_results_df = pd.concat(all_validation_results)

# 存儲到 Excel 檔案
excel_path = f'/home/b10173209/test.file/X/validation_results_lr_{learning_rate}_md_{max_depth}_ne_{n_estimators}.xlsx'
all_validation_results_df.to_excel(excel_path, index=False)



##################################################################################
##     Start Running learning_rate : 0.1、max_depth : 9、n_estimators : 300
################################################################################## 
資料夾路徑: /home/b10173209/test.file/SS/class10(KFold)/anger/anger_0
折數: 1/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 2/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 3/5


...

/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 4/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


折數: 5/5


/home/b10173209/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
